### This notebook walks you through how to use the pretrained model to generate your own transition state guesses. If using your own model and data, replace the model and data paths with your own

# THIS ONE!

In [1]:
from rdkit import Chem, Geometry
from rdkit.Chem.Draw import IPythonConsole 

import tensorflow as tf
from model.G2C import G2C
# from G2C import G2C

import numpy as np
import py3Dmol

from av_utils import prepare_batch, sample_batch

C:\Users\Avish\miniconda3\envs\tsir-rdkit-env1.1\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Avish\miniconda3\envs\tsir-rdkit-env1.1\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Avish\miniconda3\envs\tsir-rdkit-env1.1\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Avish\miniconda3\envs\tsi

## Standard

Read in the test data with rdkit. You can optionally change this to use your own file type (instead of sdf), as long as rdkit can read in the data without sanitization or hydrogen removal. Note that the reactants and prodcuts defined in the sdf **MUST** preserve atom ordering between them! Define the saved model path

In [2]:
model_path = 'log/2layers_256hs_3its/best_model.ckpt' # saves model path here
reactant_file = 'data/test_reactants.sdf'
product_file = 'data/test_products.sdf'

test_data = [Chem.ForwardSDMolSupplier(reactant_file, removeHs=False, sanitize=False),
             Chem.ForwardSDMolSupplier(product_file, removeHs=False, sanitize=False)]
# only keeps if not null
test_data = [(x,y) for (x,y) in zip(test_data[0], test_data[1]) if (x,y)] # len=842

Batch preparation code. You can use larger batch sizes if you have many predictions to speed up the predictions

In [5]:
## testing tensor stuff
# MAX_SIZE = number of atoms in reactan with most atoms
# batch_mols = test_data[i*BATCH_SIZE:(i+1)*BATCH_SIZE]  -> just batch size or however many left at end
# size = len(batch_mols) i.e. number of batch molecules
# l = np.zeros((2, 3, 4, 5), dtype=np.float32)
# l

## getting atom positions
# Chem.Get3DDistanceMatrix(test_data[0][1])
# test_data[0][1].GetAtomWithIdx(0)


In [3]:
# note: this is running the trained model on (batched) test data
# so i will use the same format: model.py and train.py for my model; then run tests in notebook

BATCH_SIZE = 1
MAX_SIZE = max([x.GetNumAtoms() for x,y in test_data]) # number of atoms in reactant with most atoms 
elements = "HCNO"
num_elements = len(elements)

# this func builds edge and vertex features for each molecule as part of a batch
def prepare_batch(batch_mols):

    # Initialization
    size = len(batch_mols) 
    V = np.zeros((size, MAX_SIZE, num_elements+1), dtype=np.float32) # vertices  [MAX_SIZE[5]]
    E = np.zeros((size, MAX_SIZE, MAX_SIZE, 3), dtype=np.float32) # leftmost number in tuple is outermost array
                                                                  # 3 because 3 features: aromatic, bonded, exp(avg dist)
                                                                  # batch index * max atoms * max atoms * 3 edge features
    sizes = np.zeros(size, dtype=np.int32) # populated later on, corresponds to each ts
    coordinates = np.zeros((size, MAX_SIZE, 3), dtype=np.float32) # number of mols in batch * max number of atoms * 3 i.e. (xyz) for each atom in mol

    # Build atom features
    for bx in range(size): # iterate through batch? yes, bx is batch index
        reactant, product = batch_mols[bx] 
        N_atoms = reactant.GetNumAtoms()
        sizes[bx] = int(N_atoms)  # cast to int [can it not be an int?], but basically have each size value as int of number atoms corresponding to reactant

        # topological distances matrix i.e. number of bonds between atoms in mol e.g. molecule v1-v2-v3-v4 will have tdm[1][4]=3
        # also symm matrix
        MAX_D = 10. # i.e. don't have more than 10 bonds between molecules
        D = (Chem.GetDistanceMatrix(reactant) + Chem.GetDistanceMatrix(product)) / 2
        D[D > MAX_D] = MAX_D 

        D_3D_rbf = np.exp( -( (Chem.Get3DDistanceMatrix(reactant) + Chem.Get3DDistanceMatrix(product) ) / 2) )  # lP: squared. AV: [is it?]
        # distance matrix between atoms aka topographic distance matrix aka geometric distance matrix
        # just averaging the distances corresponding to the same atom pairs in reactant and product

        for i in range(N_atoms):
            # Edge features
            for j in range(N_atoms):
                E[bx, i, j, 2] = D_3D_rbf[i][j]
                if D[i][j] == 1.:  # if stays bonded
                    if reactant.GetBondBetweenAtoms(i, j).GetIsAromatic():
                        E[bx, i, j, 0] = 1.
                    E[bx, i, j, 1] = 1. 
                    # so each reaction (reactant-product pair) has 3 features: whether aromatic, whether bond broken/formed, exp(avg dist)

            # Recover coordinates
            # for k, mol_typ in enumerate([reactant, ts, product]):
            pos = reactant.GetConformer().GetAtomPosition(i) 
            np.asarray([pos.x, pos.y, pos.z])
            coordinates[bx, i, :] = np.asarray([pos.x, pos.y, pos.z]) # bx is basically mol_id or rxn_id; each molecule has i atoms with (xyz)

            # Node features: whether HCNO present and then atomic number/10
            atom = reactant.GetAtomWithIdx(i) # get type of atom
            e_ix = elements.index(atom.GetSymbol()) # get chem symbol of atom and corresponding elements index
            V[bx, i, e_ix] = 1. # whether HCNO present
            V[bx, i, num_elements] = atom.GetAtomicNum() / 10. # atomic number/10

    batch_dict = {
        "nodes": V,
        "edges": E,
        "sizes": sizes,
        "coordinates": coordinates
    }
    return batch_dict, batch_mols


def sample_batch():
    batches = (len(test_data) - 1) // BATCH_SIZE + 1 # number of batches = (842-1)//(1+1) = 420
    for i in range(batches):
        batch_mols = test_data[i*BATCH_SIZE:(i+1)*BATCH_SIZE]
        yield prepare_batch(batch_mols)

Initialize the model. The hyperparameters should match those of the previously trained model (pls ignore all the deprecation warnings :) )

In [4]:
model = G2C(
      max_size=MAX_SIZE, node_features=num_elements+1, edge_features=3, layers=2, hidden_size=256, iterations=3
)



Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor



Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
Use tf.print instead of tf.Print. Note that tf.print returns a no-output operator that directly prints the output. Outside of defuns or eager mode, this operator will not be executed unless it is directly specified in session.run or used as a control dependency for other operators. This is only a concern in graph mode. Below is an example of how to ensure tf.print executes in graph mode:
```python
    sess = tf.compat.v1.Session()
    with sess.as_default():
        tensor = tf.range(10)
        print_op = tf.print(tensor)
        with tf.control_dependencies([print_op]):
          out = tf.add(tensor, tensor)
        sess.run(out)
    ```
Additionally, to u

Load the trained model and predict transition state geometries!

In [5]:
# Launch session
config = tf.ConfigProto(
    allow_soft_placement=True,
    log_device_placement=False
)

saved_d = []
with tf.Session(config=config) as sess:
    
    # Initialization
    print("Model loading...")
    saver = tf.train.Saver()
    saver.restore(sess, model_path) # restore best model
    print("Model restored")
    
    # Generator for test data
    get_test_data = sample_batch()

    X = np.empty([len(test_data), MAX_SIZE, 3])

    new_D_init = np.empty([len(test_data), MAX_SIZE, MAX_SIZE])
    new_W = np.empty([len(test_data), MAX_SIZE, MAX_SIZE])
    
    for step, data in enumerate(get_test_data):
        
        batch_dict_test, batch_mols_test = data
        feed_dict = {model.placeholders[key]: batch_dict_test[key] for key in batch_dict_test}
        X[step*BATCH_SIZE:(step+1)*BATCH_SIZE, :, :] = sess.run([model.tensors["X"]], feed_dict=feed_dict)[0]
        new_D_init[step*BATCH_SIZE:(step+1)*BATCH_SIZE, :, :] = sess.run([model.tensors["D_init"]], feed_dict=feed_dict)[0]
        # print(sess.run([model.tensors["D_init"]], feed_dict=feed_dict))

    #print(sess.run(model.tensors['D_init']))            
    #sess.run(W)
    #saved_d.append(model.tensors['D_init'])
    #saved_d.append(X)

Model loading...
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from log/2layers_256hs_3its/best_model.ckpt
Model restored


In [6]:
model.tensors

{'V': <tf.Tensor 'GraphNeuralNet/Iteration2/NodeUpdate/add:0' shape=(?, 21, 256) dtype=float32>,
 'E': <tf.Tensor 'GraphNeuralNet/Iteration2/EdgeUpdate/add:0' shape=(?, 21, 21, 256) dtype=float32>,
 'embedding': <tf.Tensor 'DistancePredictions/Sum:0' shape=(?, 256) dtype=float32>,
 'D_init': <tf.Tensor 'DistancePredictions/mul:0' shape=(?, 21, 21) dtype=float32>,
 'W': <tf.Tensor 'DistancePredictions/Softplus_1:0' shape=(?, 21, 21) dtype=float32>,
 'X': <tf.Tensor 'Loss/RMSD/transpose:0' shape=(?, 21, 3) dtype=float32>}

In [12]:
tf_params_base_folder = r'loose-files/'
# np.save('/loose-files/new_D_init.npy', new_D_init)
# model.__dict__
# model.tensors
# D_init = np.load('D_init.npy')
masked_raw_weights = np.load(tf_params_base_folder + 'masked_raw_weights.npy')
# atom_importance_weights = np.load('atom_importance_weights.npy')

# what = new_saver.restore(sess, 'log/2layers_256hs_3its/last_model.ckpt')
# all_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)

# tf.train.list_variables('log/2layers_256hs_3its/best_model.ckpt')

## With other ckpts

In [2]:
reactant_file = 'data/test_reactants.sdf'
product_file = 'data/test_products.sdf'

test_data = [Chem.ForwardSDMolSupplier(reactant_file, removeHs=False, sanitize=False),
             Chem.ForwardSDMolSupplier(product_file, removeHs=False, sanitize=False)]
# only keeps if not null
test_data = [(x,y) for (x,y) in zip(test_data[0], test_data[1]) if (x,y)] # len=842

elements = "HCNO"
num_elements = len(elements)
BATCH_SIZE = 1
MAX_SIZE = max([x.GetNumAtoms() for x,y in test_data]) # number of atoms in reactant with most atoms 
# this is same in test and train data

### Make sure G2C params from training run and here in test match
- parser.add_option("--layers", dest="layers", default=2)
- parser.add_option("--hidden_size", dest="hidden_size", default=128)
- parser.add_option("--iterations", dest="iterations", default=3)
- parser.add_option("--epochs", dest="epochs", default=200)
- parser.add_option("--batch_size", dest="batch_size", default=8)

G2C(max_size=max_size, node_features=num_elements + 1, edge_features=3,
    layers=layers, hidden_size=hidden_size, iterations=iterations, input_data=next_element)

layers, hidden_size, iterations are the hps that can change.

I need to be aware of batch_size too for recreating $D_{init}$ and $W$.

In [3]:
# make model
model = G2C(max_size=MAX_SIZE, node_features=num_elements+1, edge_features=3, layers=2, hidden_size=128, iterations=3)



Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor



Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


Instructions for updating:
Use tf.print instead of tf.Print. Note that tf.print returns a no-output operator that directly prints the output. Outside of defuns or eager mode, this operator will not be executed unless it is directly specified in session.run or used as a control dependency for other operators. This is only a concern in graph mode. Below is an example of how to ensure tf.print executes in graph mode:
```python
    sess = tf.compat.v1.Session()
    with sess.as_default():
        tensor = tf.range(10)
        print_op = tf.print(tensor)
        with tf.control_dependencies([print_op]):
          out = tf.add(tensor, tensor)
        sess.run(out)
    ```
Additionally, to u

In [10]:
model_path = 'log/21Jul03_1253PM/'
final_model_path = model_path + 'best_model.ckpt'
# model_path = 'log/21May22_0910AM/best_model.ckpt' 
# model_path = 'log/2layers_256hs_3its/best_model.ckpt' # saves model path here

# Launch session
config = tf.ConfigProto(allow_soft_placement=True, log_device_placement=False)
with tf.Session(config=config) as sess:
    
    # Initialization
    print("Model loading...")
    saver = tf.train.Saver()
    saver.restore(sess, final_model_path) # restore best model
    print("Model restored")
    
    # Generator for test data
    get_test_data = sample_batch(test_data, BATCH_SIZE, MAX_SIZE)

    X = np.empty([len(test_data), MAX_SIZE, 3])

    new_D_init = np.empty([len(test_data), MAX_SIZE, MAX_SIZE])
    new_W = np.empty([len(test_data), MAX_SIZE, MAX_SIZE])
    
    for step, data in enumerate(get_test_data):
        batch_dict_test, batch_mols_test = data
        feed_dict = {model.placeholders[key]: batch_dict_test[key] for key in batch_dict_test}
        X[step*BATCH_SIZE:(step+1)*BATCH_SIZE, :, :] = sess.run([model.tensors["X"]], feed_dict=feed_dict)[0]
        new_D_init[step*BATCH_SIZE:(step+1)*BATCH_SIZE, :, :] = sess.run([model.tensors["D_init"]], feed_dict=feed_dict)[0]

Model loading...
INFO:tensorflow:Restoring parameters from log/21Jul03_1253PM/best_model.ckpt
Model restored


In [5]:
X.shape, new_D_init.shape

((842, 21, 3), (842, 21, 21))

In [15]:
np.save(model_path + 'D_init.npy', new_D_init)
np.save(model_path + 'coords.npy', X)
# need to save weights and embeddings if poss too

## Other MIT stuff

Convert geometries into rdkit mol objects and save the geometries as an sdf

In [7]:
ts_mols = []
for bx in range(X.shape[0]):
    
    # Make copy of reactant
    mol_target = test_data[bx][0]
    mol = Chem.Mol(mol_target)

    for i in range(mol.GetNumAtoms()):
        x = X[bx, i, :].tolist()
        mol.GetConformer().SetAtomPosition(
            i, Geometry.Point3D(x[0], x[1], x[2])
        )
    ts_mols.append(mol)


model_ts_file = 'data/model_ts.sdf'
ts_writer = Chem.SDWriter(model_ts_file)
for i in range(len(ts_mols)):
    ts_writer.write(ts_mols[i])

Visualize the results. Change n to see different combinations of reactants, transition states, and products. Note that, for the TS, rdkit will add bonds based on the reactant. We'll clean this to only include common bonds between reactants and products

In [8]:
def clean_ts(mols):
    
    r_mol, ts_mol, p_mol = mols
    r_bonds = [(bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()) for bond in r_mol.GetBonds()]
    r_bonds = [tuple(sorted(b)) for b in r_bonds]
    p_bonds = [(bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()) for bond in p_mol.GetBonds()]
    p_bonds = [tuple(sorted(b)) for b in p_bonds]
    common_bonds = list(set(r_bonds) & set(p_bonds))
    
    emol = Chem.EditableMol(ts_mol)
    for bond in ts_mol.GetBonds():
        bond_idxs = tuple(sorted((bond.GetBeginAtomIdx(), bond.GetEndAtomIdx())))
        if bond_idxs not in common_bonds:
            emol.RemoveBond(bond_idxs[0], bond_idxs[1])
            emol.AddBond(bond_idxs[0], bond_idxs[1])
    return r_mol, emol.GetMol(), p_mol


def show_mol(mol, view, grid):
    mb = Chem.MolToMolBlock(mol)
    view.removeAllModels(viewer=grid)
    view.addModel(mb,'sdf', viewer=grid)
    view.setStyle({'model':0},{'stick': {}}, viewer=grid)
    view.zoomTo(viewer=grid)
    return view

In [9]:
n=1
mols = [test_data[n][0], ts_mols[n], test_data[n][1]]
view_mols = clean_ts(mols)

view = py3Dmol.view(width=960, height=500, linked=False, viewergrid=(1,3))
for i in range(3):
    show_mol(view_mols[i], view, grid=(0, i))
view.render()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [13]:
# ran: pip3 install iodata
from iodata import load_one
# nature data: gaussian log files (.log) and .qchemlog files not working
# von rudorff data: working for .xyz -> need to figure out data to use

mol = load_one('00.xyz')
mol